In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

In [32]:
# Load the files
monthly_income_by_occupation_grp = pd.read_excel("monthly_income_by_occupation_grp.xlsx")
synthetic_data = pd.read_csv("../synthetic_distribution.csv")
percentage_chg_income_by_age_grp = pd.read_excel("percentage_chg_income_by_age_grp.xlsx")

In [52]:
# For each occupation, generate synthetic monthly income based on the log-normal or normal distribution using the provided percentiles.
# Then apply the age adjustment factor to the generated monthly income.

# Extract SSOC codes from the synthetic data
synthetic_data['SSOC (2 digits)'] = synthetic_data['Occupation'].apply(
    lambda x: 0 if x == 'X1 - X5 Workers Not Elsewhere Classified' 
                else int(x.split()[0])
    )

# Map the age-based adjustment from percentage_chg_income_by_age_grp using the Age in synthetic_data
# Create bins to map age groups
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, 100]
age_grp_labels = ['15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 & Over']
synthetic_data['Age Group'] = pd.cut(synthetic_data['Age'], bins=age_bins, labels=age_grp_labels, right=True)

# Merge synthetic_data with monthly_income_by_occupation_grp on the SSOC (2 digits) to get the income percentiles
merged_data = pd.merge(synthetic_data, monthly_income_by_occupation_grp, on='SSOC (2 digits)', how='left')

# Merge synthetic data with percentage_chg_income_by_age_grp on the Age Group to get age-based adjustment
merged_data = pd.merge(merged_data, percentage_chg_income_by_age_grp[['Age Group', 'Income Adjustment']], on='Age Group', how='left')

# Function to generate random income based on log-normal distribution (using 25th, median, and 75th percentile)
def generate_income(row):
    q1 = row['Average of Gross - 25th Percentile ($)']
    q2 = row['Average of Gross - Median  ($)']
    q3 = row['Average of Gross - 75th Percentile ($)']

    # If no income data available, return NaN
    if pd.isna(q1) or pd.isna(q2) or pd.isna(q3):  
        return None
    sigma = (np.log(q3) - np.log(q1)) / 2
    mu = np.log(q2)

    # Create a truncated normal distribution that corresponds to the log-normal distribution
    min_income = 20000 # Set minimum monthly income threshold
    min_log_income = np.log(min_income)
    # Calculate the a and b parameters for truncation
    a = (min_log_income - mu) / sigma  # lower bound
    b = np.inf  # no upper bound
    np.random.seed(42)  # For reproducibility
    # Generate truncated normal samples
    truncated_normal_sample = stats.truncnorm.rvs(a=a, b=b, loc=mu, scale=sigma, size=1)
    
    # Transform the normal samples back to the log-normal scale
    return np.exp(truncated_normal_sample)

# Apply the generate_income function
merged_data['Monthly Income'] = merged_data.apply(generate_income, axis=1)

# Adjust income based on age adjustment factor
merged_data['Age Adjusted Monthly Income'] = merged_data['Monthly Income'] * merged_data['Income Adjustment']

# Convert monthly income to annual income by multiplying by 12
merged_data['Age Adjusted Annual Income'] = merged_data['Age Adjusted Monthly Income'] * 12


In [ ]:
merged_data[['Age', 'Occupation', 'Average of Gross - 75th Percentile ($)', 'Income Adjustment', 'Monthly Income', 'Age Adjusted Monthly Income', 'Age Adjusted Annual Income']]
